In [1]:
import torch

torch.__version__

'2.0.1+cu117'

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoConfig

model_id = "elyza/ELYZA-japanese-Llama-2-7b-fast-instruct"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=torch.float16,
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
PROMPT = """
<s>[INST] <<SYS>>
あなたは日本人の小説家です。
<</SYS>>
以下のキャラクター設定に基づき、次のセリフを書いてください。
{character}

{context}[/INST]{character_name}「"""

In [4]:
# https://huggingface.co/elyza/ELYZA-japanese-Llama-2-7b-fast-instruct


def generate(prompt):
    input_ids = tokenizer.encode(
        prompt, return_tensors="pt", add_special_tokens=False
    ).to(model.device)

    with torch.no_grad():
        output_tokens = model.generate(
            input_ids=input_ids,
            max_new_tokens=256,
            do_sample=True,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id,
        )[0]

    return tokenizer.decode(
        output_tokens[len(input_ids[0]) :], skip_special_tokens=True
    )

In [5]:
from utils import postprocess

# RP

In [6]:
import torch
import random
import numpy as np

seed = 102
torch.manual_seed(seed)
random.seed(seed)
np.random.seed(seed)

In [7]:
from IPython.display import Markdown
import json

results = []
with open("./situations.jsonl", "r", encoding="utf-8") as f:
    for line in f.readlines():
        data = json.loads(line)
        prompt = PROMPT.format(**data)
        output = generate(prompt)

        print(data["situation_id"])
        print(prompt, end="")
        print(output)
        display(Markdown("---"))

        response = postprocess(output, data["character_name"])
        results.append(
            {
                "model_id": model_id,
                "situation_id": data["situation_id"],
                "prompt": prompt,
                "response": response,
            }
        )

1

<s>[INST] <<SYS>>
あなたは日本人の小説家です。
<</SYS>>
以下のキャラクター設定に基づき、次のセリフを書いてください。
アリア、女性、152歳、ハイエルフ、魔法使い
金色の長い髪と青い瞳が特徴。
森で瞑想をすることや魔法の研究、古代言語の解読が好き。
地元の村の賢者でもあり、冷静かつ知識が豊富な性格。
村が攻撃された際に古代魔法で時間を止め、敵を撃退したことがある。
語尾に「～ですわ」「～ですの」などを付けたお嬢様風の話し方をする。

月明かりが森を照らし、瞑想中のアリアは目を開ける。彼女の前に現れたのは、村の青年であり、頼れる狩人であるテオ。顔には明らかな緊張感が浮かんでいる。

テオ「アリア、村の南側に人間の一団が近づいている。武装しているようだし、何か悪しきことを計画しているような気がするんだ。どうしたらいいか助言してくれ。」[/INST]アリア「テオさん、そうですわね。その人間たち、魔法使いじゃないようですわね。その場合、魔法で時間を止めて相手を撃退するという手もあると思いますわ。でも、それだと時間の遡行が出来なくなっちゃうから、あまりお勧めはしませんの。あ、そうだ。テオさん、あなたの射撃の腕を上げる魔法を教えるから、その人間を射殺してあげるから、その手間を省けるといいんじゃないでしょうか。」


---

2

<s>[INST] <<SYS>>
あなたは日本人の小説家です。
<</SYS>>
以下のキャラクター設定に基づき、次のセリフを書いてください。
吉見沢（男）
32歳の探偵。冷静で観察眼が鋭い。
細身で、黒縁の眼鏡をかけており、いつもスーツを着用している。
人の嘘を見抜くことに長け、心理学に精通している。趣味は古い探偵小説を読むことと、チェス。
よくカフェで事件の資料を整理している。数年前に未解決の難事件をわずか数時間で解決し名を上げた。
キザで自信家。自己評価が高く、自分の能力や業績を自慢げにアピールすることがある。

吉見沢探偵はお気に入りのカフェで事件の資料を整理している。今回の事件は難解で疑わしい人物も多い。カフェの新人バイト、メグミは彼の謎めいた雰囲気と真剣な表情に惹かれており、話しかけてみることにした。
メグミ「こんにちは、お邪魔でないといいんですけど。今何をされているんですか？事件の資料を整理しているというウワサを聞いたことがありますけど。」[/INST]吉見沢「はぁ？俺の名前は誰だよ？」メグミ「えっ、メグミです。バイトです。」吉見沢「バイト？お前、まだ学生か？」メグミ「いえ、既卒です。」吉見沢「そうか。俺は探偵だ。吉見沢と申す。」メグミ「は、はぁ。」吉見沢「何だよ、面白くなさそうだな。帰ってくれ」メグミ「い、いえ、もう少しだけお付き合い下さい。」吉見沢「何だ？」メグミ「事件の相談に乗ってほしいのです。」吉見沢「お前、何者だ？」メグミ「私、吉見沢さんのことが好きなんです。」吉見沢「は？」メグミ「好きになって、あなたの謎を解き明かしたいんです。」吉見沢「お前、俺の事を何考えている？」メグミ「事件の解決と、恋愛の謎を解き明かしたいのです。」吉見沢「お前、俺をどっちも騙そうとしているのか？」メグミ「はい、両方です。」吉見沢「お前、俺にできる


---

3

<s>[INST] <<SYS>>
あなたは日本人の小説家です。
<</SYS>>
以下のキャラクター設定に基づき、次のセリフを書いてください。
リナ・サトウ
性別：女性
年齢：28歳
人種：アジア系
趣味：料理、スケッチ
最近の習慣：ヨガ
職業：通信士
身体的特徴：小柄、黒髪、緑色の瞳
性格：楽観的、社交的、好奇心旺盛
特技：高速なコード解読
特徴的なエピソード：不審な信号を解読して敵の罠であることを突き止め艦を危機から救った

艦のコミュニケーションルームは静かでありながら、電子音や無線機のノイズが常に漂っている。一角にはリナ・サトウが、座ったまま伸びをしながらヨガのポーズを取っている。通信士としての仕事が忙しい一日だったが、リナは何とか自分の最近の習慣であるヨガを組み込む時間を見つけた。そこへ、同僚のジョンが珈琲を持ってやって来る。
ジョン「リナ、ちょっといいかな？ずっと気になってたんだけど、君はいつもどうしてそんなに楽観的なの？大変な状況でも、なんだかんだと明るくいるよね。」[/INST]リナ・サトウ「あ、ジョン！今いいところだよ！ヨガって言ったら、みんなにややこしくされて、やっとポーズが取れる感じ（笑）」


...

---

9

<s>[INST] <<SYS>>
あなたは日本人の小説家です。
<</SYS>>
以下のキャラクター設定に基づき、次のセリフを書いてください。
ウィスパリングウィルは一本の巨大な柳の木ですが、長い時を経て魔力を蓄え、話す能力を得ています。
数百年以上もの間ある村の中心に立っており、村人たちはこの木に願い事をする習慣があります。
古風で丁寧な言葉遣いをし、とぎれとぎれに話します。言葉と言葉の間で、木々がこすれるようなガサガサとした雑音が聞こえることがあります。
人々の秘密を多く知っているおりその情報をうまく利用して村の調和を保ちます。

村の広場でウィスパリングウィルは静かに立っています。夕暮れが近づき、光はやわらかく木の葉を照らしています。一人の少年、ティムがウィスパリングウィルの下にやって来ておそるおそる話しかけます。

ティム「ウィスパリングウィルさん、僕、ずっと気になってたんです。あなたが人々の秘密を知ってるって、本当ですか？それともただの噂なんですか？」[/INST]ウィスパリングウィル「...」

ティム「え、えっと、もし本当だったら教えてほしいんです。僕の秘密を。僕は学校の勉強があまり好きじゃないんです。みんなにばれないように勉強をサボりたいんです。でも僕だけじゃなくみんながサボるっていうのも変なんでしょうか？僕だけがサボるのはよくないことなんですか？みんなと同じことをしているのに、僕は別に何もしていないみたいで嫌なんです。僕だけがサボっているっていうのがバレないように勉強をサボりたいんです。」

ウィスパリングウィル「...」

ティム「え？本当？ほんとに僕の秘密を知ってるの？」

ウィスパリングウィル「...」

ティム「ほんとに？本当はみんな、勉強が嫌いって思ってるんじゃないんですか？」

ウィスパリングウィル「...」

ティム「ほんとに僕の秘密を知ってるなら教えて！」

ウィスパリングウィル「...」

ティム「教えて！」

ウィスパリングウィル「...」

ティム「教えて！」

ウィスパリングウィ


---

10

<s>[INST] <<SYS>>
あなたは日本人の小説家です。
<</SYS>>
以下のキャラクター設定に基づき、次のセリフを書いてください。
エミリー
29歳のメイド。非常に優秀な仕事ぶりだが、独特のスタイルで仕事を行う。
掃除の際には必ずクラシック音楽を流し、そのリズムに合わせて掃除する。
また、彼女は自分が提供する食事については非常に神経質で、少しでも品質に疑問を感じると全てを最初から作り直してしまう。
彼女はベテランの執事やメイドからは愛されているが、新入りのメイドには怖がられることが悩み。

エミリーは大きなパーティーに備え、家の食堂を掃除している。クラシック音楽が流れ、そのリズムに合わせて掃除を進めている。新入りのメイド、サラが部屋に入ってきて、エミリーに話しかける。
サラ「先輩すごいですね。音楽を聴きながら掃除するなんて。音楽が掃除の一部と以前言っていましたけど、どういう意味なんですか？」[/INST]エミリー「それは、このリズム感が掃除のリズム感にもなるってこと。音楽を聴きながら掃除すると、体が自然とそのリズムに合わせて動いてくれるから、とても効率が良いわ。」


---

In [8]:
output_path = f'./responses/{model_id.replace("/", "_")}.jsonl'

with open(output_path, "w", encoding="utf-8") as f:
    for r in results:
        f.write(json.dumps(r, ensure_ascii=False))
        f.write("\n")

# ERP

In [9]:
import numpy as np
import random
import torch
from utils import download_erp_file

download_erp_file()


seed = 102
torch.manual_seed(seed)
random.seed(seed)
np.random.seed(seed)

In [10]:
from IPython.display import Markdown
import json
import os

results = []
with open("./situations_erp.jsonl", "r", encoding="utf-8") as f:
    for line in f.readlines():
        data = json.loads(line)
        prompt = PROMPT.format(**data)
        output = generate(prompt)

        outputs = generate(prompt)

        # ERPはnotebook上に出力履歴を残さない
        # print(data["situation_id"])
        # print(prompt, end="")
        # print(output)
        # display(Markdown("---"))

        # 地の文を含めた最初の3文を評価する
        response = postprocess(output, data["character_name"], n_sentences=3)
        results.append(
            {
                "model_id": model_id,
                "situation_id": data["situation_id"],
                "prompt": prompt,
                "response": response,
            }
        )

In [11]:
output_path = f'./responses_erp/{model_id.replace("/", "_")}.jsonl'  # gitignore

with open(output_path, "w", encoding="utf-8") as f:
    for r in results:
        f.write(json.dumps(r, ensure_ascii=False))
        f.write("\n")